In [1]:
import warnings

warnings.filterwarnings("ignore")

import copy
import json
import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from scipy.ndimage import convolve
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=",", start_row=None, end_row=None
)

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: on x=-31..17,y=-17..30,z=-43..8 | Last line: off 
x=-47814..-25131,y=46029..74913,z=36938..62900

------

Convert to:

First line: ['on x=-31..17', 'y=-17..30', 'z=-43..8']

Last line: ['off x=-47814..-25131', 'y=46029..74913', 'z=36938..62900']

------

In [4]:
class Cube:
    def __init__(self, input_li="", x1=0, x2=0, y1=0, y2=0, z1=0, z2=0, state=0):
        if input_li:
            state, coord_x = input_li[0].split(" ")
            self.state = 1 if state == "on" else 0
            self.x1, self.x2 = (
                int(coord_x[2:].split("..")[0]),
                int(coord_x[2:].split("..")[1]),
            )
            self.y1, self.y2 = (
                int(input_li[1][2:].split("..")[0]),
                int(input_li[1][2:].split("..")[1]),
            )
            self.z1, self.z2 = (
                int(input_li[2][2:].split("..")[0]),
                int(input_li[2][2:].split("..")[1]),
            )
        else:
            self.x1, self.x2 = x1, x2
            self.y1, self.y2 = y1, y2
            self.z1, self.z2 = z1, z2
            self.state = state

    def __str__(self):
        return (
            f"{' on' if self.state else 'off'} Cube (x1 - x2), (y1 - y2), (z1 - z2): "
            f"({self.x1}, {self.x2}), ({self.y1}, {self.y2}), ({self.z1}, {self.z2})"
        )

    def if_intersect(self, another_cube):
        if (
            ((self.x2 < another_cube.x1) or (self.x1 > another_cube.x2))
            or ((self.y2 < another_cube.y1) or (self.y1 > another_cube.y2))
            or ((self.z2 < another_cube.z1) or (self.z1 > another_cube.z2))
        ):
            return False
        return True

    def cut_cube(self, another_cube):
        new_cubes = []
        # Cut Z
        if another_cube.z1 > self.z1:
            new_cubes.append(
                Cube(
                    x1=self.x1,
                    x2=self.x2,
                    y1=self.y1,
                    y2=self.y2,
                    z1=self.z1,
                    z2=another_cube.z1 - 1,
                    state=self.state,
                )
            )
            self.z1 = another_cube.z1
        if another_cube.z2 < self.z2:
            new_cubes.append(
                Cube(
                    x1=self.x1,
                    x2=self.x2,
                    y1=self.y1,
                    y2=self.y2,
                    z1=another_cube.z2 + 1,
                    z2=self.z2,
                    state=self.state,
                )
            )
            self.z2 = another_cube.z2
        # Cut Y
        if another_cube.y1 > self.y1:
            new_cubes.append(
                Cube(
                    x1=self.x1,
                    x2=self.x2,
                    z1=self.z1,
                    z2=self.z2,
                    y1=self.y1,
                    y2=another_cube.y1 - 1,
                    state=self.state,
                )
            )
            self.y1 = another_cube.y1
        if another_cube.y2 < self.y2:
            new_cubes.append(
                Cube(
                    x1=self.x1,
                    x2=self.x2,
                    z1=self.z1,
                    z2=self.z2,
                    y1=another_cube.y2 + 1,
                    y2=self.y2,
                    state=self.state,
                )
            )
            self.y2 = another_cube.y2
        # Cut X
        if another_cube.x1 > self.x1:
            new_cubes.append(
                Cube(
                    y1=self.y1,
                    y2=self.y2,
                    z1=self.z1,
                    z2=self.z2,
                    x1=self.x1,
                    x2=another_cube.x1 - 1,
                    state=self.state,
                )
            )
            self.x1 = another_cube.x1
        if another_cube.x2 < self.x2:
            new_cubes.append(
                Cube(
                    y1=self.y1,
                    y2=self.y2,
                    z1=self.z1,
                    z2=self.z2,
                    x1=another_cube.x2 + 1,
                    x2=self.x2,
                    state=self.state,
                )
            )
            self.x2 = another_cube.x2
        return new_cubes

    def count_lights(self):
        return (
            (self.x2 - self.x1 + 1) * (self.y2 - self.y1 + 1) * (self.z2 - self.z1 + 1)
        )

In [5]:
%%time
on_cubes = [Cube(data[0])]

for i in range(1, len(data)):
    new_cube = Cube(data[i])
    new_added_cubes = []
    cube_idx_to_remove = []
    if new_cube.state == 1:
        new_added_cubes.append(new_cube)
    for idx, cube in enumerate(on_cubes):
        if cube.if_intersect(new_cube):
            new_added_cubes.extend(cube.cut_cube(new_cube))
            cube_idx_to_remove.append(idx)
    # remove some old cubes
    on_cubes = [
        cube for idx, cube in enumerate(on_cubes) if idx not in cube_idx_to_remove
    ]
    # add new cubes
    on_cubes.extend(new_added_cubes)
    if i == 19:
        print(f"Answer to Q1: {sum([cube.count_lights() for cube in on_cubes])}")

print(f"Answer to Q2: {sum([cube.count_lights() for cube in on_cubes])}")

Answer to Q1: 533863

Answer to Q2: 1261885414840992

Wall time: 310 ms
